# Hyperparameter Tuning

## Calibrating the learning rate of the Gradient Descent optimiser

In this notebook, we calibrate the learning rate for the gradient descent optimiser on a parameter identification problem.

### Setting up the Environment

If you don't already have PyBOP installed, check out the [installation guide](https://pybop-docs.readthedocs.io/en/latest/installation.html) first.

We begin by importing the necessary libraries. Let's also fix the random seed to generate consistent output during development.

In [ ]:
import numpy as np
import pybamm

import pybop

pybop.plot.PlotlyManager().pio.renderers.default = "notebook_connected"

np.random.seed(8)  # users can remove this line

## Generating a synthetic dataset

To demonstrate parameter estimation, we first need some data. We will generate synthetic data using the single particle model (SPM) as the forward model, which requires defining the parameter values and the model itself.

In [ ]:
model = pybamm.lithium_ion.SPM()
parameter_values = pybamm.ParameterValues("Chen2020")
parameter_values.update(
    {
        "Negative electrode active material volume fraction": 0.65,
        "Positive electrode active material volume fraction": 0.51,
    }
)
parameter_values.set_initial_state(0.4)
experiment = pybamm.Experiment(
    [
        "Discharge at 0.5C for 6 minutes (5 second period)",
        "Charge at 0.5C for 6 minutes (5 second period)",
    ]
    * 2
)
solution = pybamm.Simulation(
    model, parameter_values=parameter_values, experiment=experiment
).solve()

To make the parameter estimation more realistic, we add Gaussian noise to the data. The dataset for optimisation is composed of time, current, and the noisy voltage data:

In [ ]:
sigma = 0.002  # 2 mV
corrupt_values = solution["Voltage [V]"].data + np.random.normal(
    0, sigma, len(solution.t)
)
dataset = pybop.Dataset(
    {
        "Time [s]": solution.t,
        "Current function [A]": solution["Current [A]"].data,
        "Voltage [V]": corrupt_values,
    }
)

## Identifying the parameters

We select the parameters for estimation and set up their prior distributions and bounds:

In [ ]:
parameters = {
    "Negative electrode active material volume fraction": pybop.Parameter(
        initial_value=0.5,
    ),
    "Positive electrode active material volume fraction": pybop.Parameter(
        initial_value=0.5,
    ),
}
true_values = [parameter_values[p] for p in parameters.keys()]
parameter_values.update(parameters)

### Setting up the problem with an unsuitable sigma value

With the datasets and parameters defined, we can set up the optimisation problem, its cost function, and the optimiser. Let's begin by setting the learning rate (`eta`) to be quite small.

In [ ]:
simulator = pybop.pybamm.Simulator(
    model, parameter_values=parameter_values, protocol=dataset
)
cost = pybop.SumSquaredError(dataset)
problem = pybop.Problem(simulator, cost)
options = pybop.PintsOptions(max_iterations=100)
optim = pybop.GradientDescent(problem, options=options)
optim.optimiser.set_learning_rate(eta=0.01)

NOTE: Boundaries ignored by <class 'pybop.optimisers._gradient_descent.GradientDescentImpl'>


We proceed to run the optimiser with the updated learning rate. After the optimisation, we can examine the estimated parameter values. In this case, the optimised values differ from the ground truth values.

In [ ]:
result = optim.run()

print("True values:", true_values)
print("Estimates:", result.x)

True values: [0.65, 0.51]
Estimates: [0.61779841 0.5457801 ]


## Calibrating the learning rate 

Now that we've seen how an unsuitable `eta` value prevents the optimiser from converging within the maximum number of iterations, let's calibrate this value to find the optimal solution using fewer iterations.

In [ ]:
etas = np.linspace(0.02, 0.42, 3)
results = []
for eta in etas:
    optim = pybop.GradientDescent(problem, options=options)
    optim.optimiser.set_learning_rate(eta)
    result = optim.run()
    results.append(result)

for result, eta in zip(results, etas, strict=False):
    print(
        f"| eta: {eta} | Num Iterations: {result.n_iterations} | Best Cost: {result.best_cost} | Results: {result.x} |"
    )

NOTE: Boundaries ignored by <class 'pybop.optimisers._gradient_descent.GradientDescentImpl'>


NOTE: Boundaries ignored by <class 'pybop.optimisers._gradient_descent.GradientDescentImpl'>


NOTE: Boundaries ignored by <class 'pybop.optimisers._gradient_descent.GradientDescentImpl'>


| eta: 0.02 | Num Iterations: 100 | Best Cost: 0.00148834298963961 | Results: [0.63087175 0.53068887] |
| eta: 0.21999999999999997 | Num Iterations: 36 | Best Cost: 0.0012809594064389588 | Results: [0.6493396  0.51153732] |
| eta: 0.42 | Num Iterations: 19 | Best Cost: 0.0015287449704297902 | Results: [0.64197256 0.52952851] |


From these results, we can see that `eta=0.22` returns the best cost value by balancing fast convergence with a small enough step size to avoid jumping over fine changes in the landscape.

### Cost landscapes

Another way to view this information is to plot the optimiser trace on the cost landscape.

In [ ]:
# Plot the cost landscape with optimisation path and updated bounds
bounds = np.array([[0.4, 0.8], [0.4, 0.8]])
for result, eta in zip(results, etas, strict=False):
    result.plot_surface(bounds=bounds, title=f"eta: {eta}")

### Optimised time-series trajectory

Let's take `eta = 0.22` as the best learning rate for this problem and look at the time-series trajectory.

In [ ]:
pybop.plot.problem(
    problem, inputs=results[2].best_inputs, title="Optimised Comparison"
);

## Concluding thoughts

This notebook covers how to calibrate the learning rate for the gradient descent optimiser, thus providing an introduction to hyperparameter tuning.

We have shown how a small learning rate impedes the optimiser by imposing a small step size that requires many iterations to traverse the search space. A larger learning rate can provide the best performance, but too large a learning rate can cause the gradient descent algorithm to diverge (as shown in the last cost landscape plot).